# **CS 1810 Homework 1**
---
To account for potential version issues, try the following in your terminal:

1. Create a new environment with `python3 -m venv venv`
2. Activate that environment with `source venv/bin/activate`
3. Make sure the interpreter in the top right corner of your VSCode (or whatever you use to run your code is venv).
4. If you get a "install kernel" message, press it.
5. Run `pip install -r requirements.txt`
6. Run the remainder of this notebook.

Note that this is not necessary but can help prevent any issues due to package versions.

**The following notebook is meant to help you work through Problems 1, 3, and 4 on Homework 1. You are by no means required to use it, nor are you required to fill out/use any of the boilerplate code/functions. You are welcome to implement the functions however you wish.**

In [ ]:
# Loading data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = np.genfromtxt("data/earth_temperature_sampled_train.csv", delimiter = ',')
year_train = train_data[:, 0] / 1000
temp_train = train_data[:, 1]
test_data = np.genfromtxt("data/earth_temperature_sampled_test.csv", delimiter = ',')
year_test = test_data[:, 0] / 1000
temp_test = test_data[:, 1]

df_train = pd.read_csv("data/earth_temperature_sampled_train.csv")
X_train = df_train["# Age"] / 1000
y_train = df_train[" Temperature"]

df_test = pd.read_csv("data/earth_temperature_sampled_test.csv")
X_test = df_train["# Age"] / 1000
y_test = df_train[" Temperature"]

print(f"First 5 X Training (# Age):\n{X_train.head()}\n")
print(f"First 5 y Training (Temperature):\n{y_train.head()}")
print(f"  Mean:     {y_train.mean():.2f}")
print(f"  Median:   {y_train.median():.2f}")
print(f"  Std Dev:  {y_train.std():.2f}")
print(f"  Min:      {y_train.min():.2f}")
print(f"  Max:      {y_train.max():.2f}")
print(f"  Range:    {y_train.max() - y_train.min():.2f}")
print(f"  Q1 (25%): {y_train.quantile(0.25):.2f}")
print(f"  Q3 (75%): {y_train.quantile(0.75):.2f}")
print(f"  IQR:      {y_train.quantile(0.75) - y_train.quantile(0.25):.2f}")

In [ ]:
print("\nX_train (Age in thousands)")
print(f"Shape: {X_train.shape}")
print(f"Type: {X_train.dtype}")
print(f"  Mean:     {X_train.mean():.2f}")
print(f"  Median:   {X_train.median():.2f}")
print(f"  Std Dev:  {X_train.std():.2f}")
print(f"  Min:      {X_train.min():.2f}")
print(f"  Max:      {X_train.max():.2f}")
print(f"  Range:    {X_train.max() - X_train.min():.2f}")
print(f"  Q1 (25%): {X_train.quantile(0.25):.2f}")
print(f"  Q3 (75%): {X_train.quantile(0.75):.2f}")
print(f"  IQR:      {X_train.quantile(0.75) - X_train.quantile(0.25):.2f}")

print("\ny_train (Temperature in K)")
print(f"Shape: {y_train.shape}")
print(f"Type: {y_train.dtype}")
print(f"  Mean:     {y_train.mean():.2f}")
print(f"  Median:   {y_train.median():.2f}")
print(f"  Std Dev:  {y_train.std():.2f}")
print(f"  Min:      {y_train.min():.2f}")
print(f"  Max:      {y_train.max():.2f}")
print(f"  Range:    {y_train.max() - y_train.min():.2f}")
print(f"  Q1 (25%): {y_train.quantile(0.25):.2f}")
print(f"  Q3 (75%): {y_train.quantile(0.75):.2f}")
print(f"  IQR:      {y_train.quantile(0.75) - y_train.quantile(0.25):.2f}")

# Problem 1

## Problem 1 Subpart 1(a)

In [ ]:
def predict_knn(x_new, k, x_train, y_train):
    """
    Returns predictions for the values in x_test, using KNN predictor with the specified k.

    :param x_new: a numpy array of x_values on which to do prediction. Shape is (n,)
    :param k: number of nearest neighbors to consider
    :param x_train: x coordinates of training dataset
    :param y_train: y coordinates of training dataset

    :return: if x_new = [x_1, x_2, ...], then return [f(x_1), f(x_2), ...]
             where f is the kNN with specified parameters and training set
    """

    # Compute distances, shape is (n, m) where n is the number of test points and m is the number of training points
    dists = np.abs(x_train.reshape(1,-1) - x_new.reshape(-1,1))
    # Argsort the rows
    ix = dists.argsort(axis = 1)
    ix = ix[:, :k] # take only the k smallest distances
    y = y_train[ix]

    # sum each row
    return np.mean(y, axis = 1)

In [ ]:
# plot functions
N = year_train.shape[0]
x_array = np.arange(400, 800, 1)
plt.plot(x_array, predict_knn(x_array, 1, year_train, temp_train), label = "$k = 1$")
plt.plot(x_array, predict_knn(x_array, 3, year_train, temp_train), label = "$k = 3$")
plt.plot(x_array, predict_knn(x_array, N - 1, year_train, temp_train), label = "$k = N - 1$")
plt.scatter(year_train, temp_train, label = "training data", color = "red")
plt.ylabel("Temperature")
plt.xlabel("Year BCE (in thousands)")

plt.legend()
plt.xticks(np.arange(400, 900, 100))
plt.ylim([-10,2.5])

plt.gca().invert_xaxis()
# save figure to img_output directory
plt.savefig("img_output/p1.1a.png", bbox_inches = "tight")
plt.show()

## Problem 1 Subpart 1(b)

In [ ]:
def model_mse(predictions, true):
    """
    Calculate the MSE for the given model predictions, with respect to the true values

    :param predictions: predictions given by the model
    :param true: corresponding true values
    :return: the mean squared error
    """
    return np.mean((predictions - true) ** 2)

In [ ]:
# Compute the MSEs for different values of k
# YOUR CODE HERE
for k in [1, 3, N - 1]:
    predictions = predict_knn(year_test, k, year_train, temp_train)
    mse = model_mse(predictions, temp_test)
    print(f"k = {k}: Test MSE = {mse:.4f}")


## Problem 1 Subpart 2(a)

In [ ]:
def kernel_regressor(x_new, tau, x_train, y_train):
    """
    Run f_tau(x) with parameter tau on every entry of x_new.

    :param x_new: a numpy array of x_values on which to do prediction. Shape is (n,)
    :param float tau: lengthscale parameter
    :param y_train: the x coordinates of the training set
    :param y_train: the y coordinates of the training set
    :return: if x_new = [x_1, x_2, ...], then return [f(x_1), f(x_2), ...]
             where f is calculated wrt to the training data and tau
    """
    dists_squared = (x_new.reshape(-1, 1) - x_train.reshape(1, -1)) ** 2
    K = np.exp(-dists_squared / tau)

    numerator = np.sum(K * y_train.reshape(1, -1), axis=1)
    denominator = np.sum(K, axis=1)

    return numerator / denominator

In [ ]:
# Plot functions
x_array = np.arange(400, 800 + 1, 1)
for tau in [1, 50, 2500]:
    plt.plot(x_array, kernel_regressor(x_array, tau, year_train, temp_train), label = f"$\\tau = {tau}$")
plt.scatter(year_train, temp_train, label = "training data", color = "red")
plt.legend()
plt.xticks(np.arange(400, 800 + 100, 100))
plt.ylabel("Temperature")
plt.xlabel("Year BCE (in thousands)")
plt.ylim([-10,2.5])

plt.gca().invert_xaxis()
# save figure to img_output directory
plt.savefig("img_output/p1.2a.png", bbox_inches = "tight")
plt.show()

## Problem 1 Subpart 2(c)

In [ ]:
# Compute the MSEs for different values of tau
# YOUR CODE HERE
for tau in [1, 50, 2500]:
    predictions = kernel_regressor(year_test, tau, year_train, temp_train)
    mse = model_mse(predictions, temp_test)
    print(f"Tau = {tau}: Test MSE = {mse:.4f}")

# Problem 3

## Problem 3 Subpart 1

In [ ]:
def exp_kernel(x,mu):
  return np.exp(-1/float(5)*np.power(x-mu,2))

def f_scale(X, part = "a"):
  if part == "a":
    X = X/181 # 181000
  elif part == "b":
    X = X/4e2 # 4e5
  elif part == "c":
    X = X/1.81 # 1810    
  elif part == "d":
    X = X/.181 # 181
  return X

# TODO: Complete this `make_basis` function according to the above
# specifications. The function should return the array `phi(X)`

def make_basis(X, part='a'):
  """
  Args:
    X: input of years (or any variable you want to turn into the appropriate basis) as
        ndarray with length `N`.
    part: one of `a`, `b`, `c`, `d` depending on the basis function.

    Returns:
      ndarray `phi(X)` of shape `(N,D)`. For each part the shapes of your
      training data `make_basis(years_train)` should be
        (a) 57x10, (b) 57x10, (c) 57x10, (d) 57x50.
  """
    
  ### DO NOT CHANGE THIS SECTION 
  ### it is to prevent numerical instability from taking the exponents of
  ### the years, as well as break symmetry when dealing with a Fourier basis.
  X = f_scale(X, part)
  ### end section

  N = len(X)
  if part == 'a':
    D = 10
    phi = np.ones((N, D))
    for j in range(1, D):
      phi[:, j] = np.power(X, j)

  elif part == 'b':
    D = 10
    phi = np.ones((N, D))
    for j in range(1, D):
      mu_j = (j + 7) / 8.0
      phi[:, j] = exp_kernel(X, mu_j)

  elif part == 'c':
    D = 10
    phi = np.ones((N, D))
    for j in range(1, D):
      phi[:, j] = np.cos(X / j)

  elif part == 'd':
    D = 50
    phi = np.ones((N, D))
    for j in range(1, D):
      phi[:, j] = np.cos(X / j)

  return phi

We are now solving the multi-dimensional OLS regression problem. For each $i=1,\ldots, N$, we have 
$$ \hat y_i = \mathbf{w}^\top\mathbf{\phi}(x_i) = \sum_{j=1}^D w_j \phi_j(x_i).  $$

We can find the weights that minimize the MSE $\frac 1N\| \mathbf{y} - \mathbf{\phi}(\mathbf{X})\mathbf{w}\| $ with the analytic solution described in the textbook at Derivation 2.6.1.
$$ \mathbf{w^*} = (\mathbf{X}^\top \mathbf{X})^{-1} \mathbf{X}^\top \mathbf{y}. $$

In [ ]:
# Helper function to find the regression weights using the Moore-Penrose pseudoinverse.
def find_weights(X,y):
    w_star = np.dot(np.linalg.pinv(np.dot(X.T, X)), np.dot(X.T, y))
    return w_star

In [ ]:
_, ax = plt.subplots(2,2, figsize = (16,10))

mse_results = {}

for i, part in enumerate(['a', 'b', 'c' ,'d']):
  # Plotting the original data
  phi_years_train = make_basis(year_train, part)
  w = find_weights(phi_years_train, temp_train)

  phi_test = make_basis(year_test, part)
  y_test_pred = phi_test @ w
  mse = model_mse(y_test_pred, temp_test)
  mse_results[part] = mse

  
  ax[i//2, i%2].scatter(year_train, temp_train, label = "Original Data")
  
  xs = np.linspace(year_train.min(), year_train.max(), 1000)
  ax[i//2, i%2].set_xlabel("Year")
  ax[i//2, i%2].set_ylabel("Temperature")
  ax[i//2, i%2].set_title(f"OLS Basis Regression; Temperature on Years ({part})")

  ax[i//2, i%2].legend()

  # TODO: Plot the regression line generated by your model. 
  # YOUR CODE HERE
  pass
  ax[i//2, i%2].invert_xaxis()
  
plt.savefig("img_output/p3.1.png")

## Problem 3 Subpart 2

In [ ]:
# Compute the MSE for each basis
# YOUR CODE HERE
for part in ['a', 'b', 'c', 'd']:
    print(f"Basis ({part}): Test MSE = {mse_results[part]:.4f}")

# Problem 4

## Problem 4 Subpart 5

In [ ]:
def find_lasso_weights(lam, X, y):
    """
    Fit the weights of a LASSO linear regression through the coordinate descent algorithm.

    :param lam: the lambda parameter
    :param X: the design matrix with training set features
    :param y: the training set labels
    :param max_iter: maximum number of iterations
    :param tol: convergence tolerance
    :return: the fitted weights
    """
    N, D = X.shape
    w = np.ones(D)  # Initialize with ones as specified
    tol = 1e-6

    for iteration in range(5000):
        w_old = w.copy()

        for d in range(D):
            # Extract the d-th column
            x_d = X[:, d]

            # Compute residual excluding the contribution of w_d
            residual = y - (X @ w - w[d] * x_d)

            # Compute rho_d
            rho_d = x_d @ residual

            # Update w_d
            if d == 0:  # Bias term (no regularization)
                w[d] = rho_d / (x_d @ x_d)
            else:
                norm_sq = x_d @ x_d
                w[d] = np.sign(rho_d) * max(abs(rho_d) - lam / 2, 0) / norm_sq

        # Check convergence
        if np.linalg.norm(w - w_old) < tol:
            print(f"   Converged after {iteration + 1} iterations")
            break

    return w

In [ ]:
# Helper function for standardizing inputs to LASSO
def preprocess_lasso(X):
    X = make_basis(X, part='d')
    X[:, 1:] = (X[:, 1:] - X[:, 1:].mean(axis = 0)) / X[:, 1:].std(axis = 0)
    return X

In [ ]:
# Fit the weights for both models
phi_x_train = preprocess_lasso(year_train)
lam1, lam2 = 1, 10
w_unreg = find_weights(phi_x_train, temp_train)
w1 = find_lasso_weights(lam1, phi_x_train, temp_train)
w2 = find_lasso_weights(lam2, phi_x_train, temp_train)

# Plot functions
x_array = np.arange(400, 800 + 1, 1)
phi_x_array = preprocess_lasso(x_array)

# TODO: Plot the regression line generated by your model. 
# YOUR CODE HERE
plt.figure(figsize=(10, 6))
plt.plot(x_array, phi_x_array @ w_unreg, label="Unregularized (Basis d)", linewidth=2)
plt.plot(x_array, phi_x_array @ w1, label=f"LASSO $\\lambda={lam1}$", linewidth=2)
plt.plot(x_array, phi_x_array @ w2, label=f"LASSO $\\lambda={lam2}$", linewidth=2)
pass

plt.scatter(year_train, temp_train, label = "training data", color = "red")
plt.legend()
plt.xticks(np.arange(400, 800 + 100, 100))
plt.ylabel("Temperature")
plt.xlabel("Year BCE (in thousands)")

plt.gca().invert_xaxis()
# save figure to img_output directory
plt.savefig("img_output/p4.5.png", bbox_inches = "tight")
plt.show()

In [ ]:
# Compute the MSE for both values of lambda
# YOUR CODE HERE
pass
#phi_test = preprocess_lasso(year_test)

mse_unreg = model_mse(phi_test @ w_unreg, temp_test)
mse_lam1 = model_mse(phi_test @ w1, temp_test)
mse_lam2 = model_mse(phi_test @ w2, temp_test)

print(f"   Unregularized: Test MSE = {mse_unreg:.4f}")
print(f"   LASSO λ={lam1}:    Test MSE = {mse_lam1:.4f}")
print(f"   LASSO λ={lam2}:   Test MSE = {mse_lam2:.4f}")